[View in Colaboratory](https://colab.research.google.com/gist/naviarh/2b61fb4170f9620c34edbd545669cdaa/fastai_rnn1.ipynb)

In [0]:
!pip install --upgrade pip
!pip3 install fastai

### Настройка платформы

In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pwd

/content


In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [4]:
import subprocess, os
os.uname()

posix.uname_result(sysname='Linux', nodename='bb0f6817e6dd', release='4.14.33+', version='#1 SMP Wed Jun 20 01:36:48 PDT 2018', machine='x86_64')

In [5]:
from fastai.structured import *
from fastai.column_data import *
!pip3 show fastai torch | grep Name -A 1

Name: fastai
Version: 0.7.0
--
Name: torch
Version: 0.3.1


In [0]:
from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [6]:
torch.cuda.set_device(0)
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
#from fastai.imports import *
#from fastai.transforms import *
#from fastai.conv_learner import *
#from fastai.model import *
#from fastai.dataset import *
#from fastai.sgdr import *
#from fastai.plots import *
#!pip3 show fastai torch | grep Name -A 1

### Начало

Мы собираемся скачать собрание сочинений Ницше в качестве данных для этого класса.

In [0]:
PATH='nietzsche/'
os.makedirs(PATH, exist_ok=True)

In [11]:
get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", f'{PATH}nietzsche.txt')
text = open(f'{PATH}nietzsche.txt').read()
print('corpus length:', len(text))

nietzsche.txt: 606kB [00:00, 1.13MB/s]                           


corpus length: 600893


In [12]:
text[:400]

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to Truth, have been unskilled and unseemly methods for\nwinning a woman? Certainly she has never allowed herself '

In [13]:
chars = sorted(list(set(text))) # отсортированный список множества всех символов
vocab_size = len(chars)+1 # количество символов (+1 для нулевого значения)
print('total chars:', vocab_size)

total chars: 85


In [42]:
len(set(text))

84

In [17]:
print(chars)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [18]:
# Вставим нулевое значение в набор для отступов
chars.insert(0, "\0")

''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxy'

In [19]:
print(chars)

['\x00', '\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [20]:
len(chars)

85

In [0]:
# Карта символов в индексы
char_indices = {c: i for i, c in enumerate(chars)}
# Карта индексов в символы
indices_char = {i: c for i, c in enumerate(chars)}

In [25]:
print(char_indices)

{'\x00': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '=': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, 'Æ': 80, 'ä': 81, 'æ': 82, 'é': 83, 'ë': 84}


In [26]:
print(indices_char)

{0: '\x00', 1: '\n', 2: ' ', 3: '!', 4: '"', 5: "'", 6: '(', 7: ')', 8: ',', 9: '-', 10: '.', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ';', 23: '=', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31: 'G', 32: 'H', 33: 'I', 34: 'J', 35: 'K', 36: 'L', 37: 'M', 38: 'N', 39: 'O', 40: 'P', 41: 'Q', 42: 'R', 43: 'S', 44: 'T', 45: 'U', 46: 'V', 47: 'W', 48: 'X', 49: 'Y', 50: 'Z', 51: '[', 52: ']', 53: '_', 54: 'a', 55: 'b', 56: 'c', 57: 'd', 58: 'e', 59: 'f', 60: 'g', 61: 'h', 62: 'i', 63: 'j', 64: 'k', 65: 'l', 66: 'm', 67: 'n', 68: 'o', 69: 'p', 70: 'q', 71: 'r', 72: 's', 73: 't', 74: 'u', 75: 'v', 76: 'w', 77: 'x', 78: 'y', 79: 'z', 80: 'Æ', 81: 'ä', 82: 'æ', 83: 'é', 84: 'ë'}


In [28]:
# Данные текста (text) в виде индексов
idx = [char_indices[c] for c in text]
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [29]:
# Пример получения строки стекста по карте индексов
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

Теперь наш датасет - переменная **idx**

### Модель трёх символов

In [33]:
# Списки каждого 4 символа
cs=3 # 
# Начиная с нулевого символа
c1_dat = [idx[i]   for i in range(0, len(idx)-cs, cs)]
# Начиная с первого символа
c2_dat = [idx[i+1] for i in range(0, len(idx)-cs, cs)]
# Начиная со второго символа
c3_dat = [idx[i+2] for i in range(0, len(idx)-cs, cs)]
# Начиная с третьего символа
c4_dat = [idx[i+3] for i in range(0, len(idx)-cs, cs)]
print(c1_dat[:5], c2_dat[:5], c3_dat[:5], c4_dat[:5])

[40, 30, 29, 1, 40] [42, 25, 1, 43, 40] [29, 27, 1, 45, 39] [30, 29, 1, 40, 43]


Преобразуем списки в массивы

In [36]:
# наши входы
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [38]:
# наш выход
y = np.stack(c4_dat)
y[:4]

array([30, 29,  1, 40])

In [40]:
x1.shape, x2.shape, x3.shape, y.shape

((200297,), (200297,), (200297,), (200297,))

### Создание и обучение модели

In [0]:
# Количество активаций скрытого слоя
n_hidden = 256

In [0]:
# Количество n_fac примерно половины словаря (количества используемых символов)
n_fac = 42

In [0]:
# Создадим класс модели
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        
        # Матрицы весов
        
        self.e = nn.Embedding(vocab_size, n_fac) # слой эмбеддинга

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        self.l_in = nn.Linear(n_fac, n_hidden) # линейный входной слой

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden) # скрытый слой
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size) # выходной слой
        
    def forward(self, c1, c2, c3):
      # Один входной слой
        in1 = F.relu(self.l_in(self.e(c1))) # эмбеддинг -> линейный -> выпрямитель'
        in2 = F.relu(self.l_in(self.e(c2))) # вычисляем для каждого входа
        in3 = F.relu(self.l_in(self.e(c3)))
      # Один скрытый слой
        h = V(torch.zeros(in1.size()).cuda()) # берёём пустую матрицу
        h = F.tanh(self.l_hidden(h+in1)) # и суммируем функцию скрытого слоя
        h = F.tanh(self.l_hidden(h+in2)) # от результатов каждого входа
        h = F.tanh(self.l_hidden(h+in3))
      # Выходной слой
        return F.log_softmax(self.l_out(h))
    
    # Для наглядности
    def forward2(self, c1, c2, c3):
      h = V(torch.zeros(in1.size()).cuda())
      i = F.relu(self.l_in(self.e(c1)))
      h = F.tanh(self.l_hidden(h+i))
      i = F.relu(self.l_in(self.e(c2)))
      h = F.tanh(self.l_hidden(h+i))
      i = F.relu(self.l_in(self.e(c3)))
      h = F.tanh(self.l_hidden(h+i))
      return F.log_softmax(self.l_out(h))
    
    # Рефакторинг
    def forward3(self, c1, c2, c3):
      h = V(torch.zeros(in1.size()).cuda())
      for _ in range(3):
        i = F.relu(self.l_in(self.e(c1)))
        h = F.tanh(self.l_hidden(h+i))
      return F.log_softmax(self.l_out(h))

In [55]:
# Используем fastai для загрузки данных
md = ColumnarModelData.from_arrays('.', [-1], np.stack([x1,x2,x3], axis=1), y, bs=512)
md

In [56]:
# Создадим модель
m = Char3Model(vocab_size, n_fac).cuda()
m

Char3Model(
  (e): Embedding(85, 42)
  (l_in): Linear(in_features=42, out_features=256, bias=True)
  (l_hidden): Linear(in_features=256, out_features=256, bias=True)
  (l_out): Linear(in_features=256, out_features=85, bias=True)
)

In [57]:
# итератор для прохождения по обучающей выборке
it = iter(md.trn_dl)
it

<generator object DataLoader.__iter__ at 0x7f474dee6e60>

In [77]:
# Минибатчи достаются таким образом:
*xs,yt = next(it)

len(xs), xs[0].size(), yt.size()

(3, torch.Size([512]), torch.Size([512]))

In [74]:
# пример вхоодных данных
# 3 строки по 512 столбцов
# вместо прямого кодирования - эмбендинги
xs[0][:5], xs[1][:5], xs[2][:5]

(
  71
  58
  54
  68
  65
 [torch.cuda.LongTensor of size 5 (GPU 0)], 
   2
  67
  72
   2
  65
 [torch.cuda.LongTensor of size 5 (GPU 0)], 
  73
   1
   2
  72
  10
 [torch.cuda.LongTensor of size 5 (GPU 0)])

In [78]:
# Модель можно вызывать как функцию, 
# передавая в неё обёрнутые в Variable входные данные.
# Возвращает в нашем случае матрицу логарифмов вероятностей 512x85
# 512 - размер минибатча
# 85 - количество признаков
t = m(*V(xs))
t

Variable containing:
-4.3962 -4.3154 -4.5211  ...  -4.3912 -4.5435 -4.4732
-4.5377 -4.4180 -4.6580  ...  -4.6044 -4.5565 -4.5929
-4.3207 -4.4384 -4.3723  ...  -4.6725 -4.5566 -4.4587
          ...             ⋱             ...          
-4.2022 -4.3583 -4.6172  ...  -4.4522 -4.5896 -4.5665
-4.2314 -4.3622 -4.4085  ...  -4.6435 -4.6685 -4.4172
-4.4942 -4.4306 -4.7413  ...  -4.5525 -4.5084 -4.3842
[torch.cuda.FloatTensor of size 512x85 (GPU 0)]

In [0]:
# Используем алгоритм оптимизации PyTorch
opt = optim.Adam(m.parameters(), 0.01)

In [80]:
# Теперь можно проводить обучение
fit(m, md, 1, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      2.092768   0.946334  



[array([0.94633])]

In [0]:
# Изменяем скорость обучения
set_lrs(opt, 0.001)

In [91]:
fit(m, md, 7, opt, F.nll_loss)

epoch      trn_loss   val_loss   
    0      1.789143   0.285271  
    1      1.758075   0.436972  
 23%|██▎       | 91/392 [00:01<00:05, 54.15it/s, loss=1.74]

    2      1.742633   0.431403  
    3      1.721344   0.436019  
 67%|██████▋   | 261/392 [00:04<00:02, 56.96it/s, loss=1.72]

    4      1.714571   0.280933  
    5      1.70305    0.351053  
 89%|████████▊ | 347/392 [00:06<00:00, 56.91it/s, loss=1.69]

    6      1.699738   0.30751   



[array([0.30751])]

### Получаем результаты

In [0]:
# Функция прогнозирования
def get_next(inp):
  # Три символа заменяются на индексы
  # и оборачиваются в массив, после в тензор
    idxs = T(np.array([char_indices[c] for c in inp]))
  # затем в Variable и передаются модели
    p = m(*VV(idxs))
  # Возвращаем самое вероятное предсказание
  # в формате numpy
    i = np.argmax(to_np(p))
    return chars[i]

Пробуем

In [84]:
get_next('and')

' '

In [85]:
get_next(' th')

'e'

In [86]:
get_next('fil')

'l'

In [0]:
# Создадим генератор текста
def gen_text(three, n=1):
  result = three
  for _ in range(n):
    simb = get_next(three)
    three = three[1:] + simb
    result += simb
  return result

In [112]:
gen_text('app', n=50)

'appense of the sting the sting the sting the sting th'

In [0]:
# Создадим генератор слов
def gen_word(three):
  result = three
  for _ in range(10):
    simb = get_next(three)
    if simb == ' ': break
    three = three[1:] + simb
    result += simb
  return result

In [115]:
gen_word('app')

'appense'

In [116]:
gen_word('wor')

'world'

In [117]:
gen_word('git')

'gition'

In [118]:
gen_word('mor')

'more'

In [119]:
gen_word('shi')

'shing'